# Introduction to NEMS

Modeling with NEMS four basic steps:
1. Loading properly formatted data and performing any necessary preprocessing.
1. Building a model for the data.
1. Fitting the model to the data.
1. Assessing model performance.

In this introduction, we'll cover the basic coding objects used to accomplish
those steps and then walk through a simple example.

### Important objects
1. `Layer`: encapsulates a single mathematical transformation and an associated
          set of trainable parameters.
1. `Model`: a collection of Layers that determines the transformation between input
          data and the predicted output.

### Layer
Most Layers have one or more trainable `Parameter` values tracked by a single
`Phi` container. These objects determine the trainable numbers that will be
substituted in the mathematical operation specified by `Layer.evaluate`.
The `Parameter` and `Phi` classes can be thought of as numpy arrays and python
dictionaries, respectfully, with some NEMS-specific bookkeeping tacked on.
Default `Parameter` values are determined by the `Layer.initial_parameters`
method.

Let's design a couple new Layer subclasses to demonstrate:

In [1]:
import numpy as np
from nems.layers.base import Layer, Phi, Parameter

class Sum(Layer):
    def evaluate(self, *inputs):
        # All inputs are treated the same, no fittable parameters.
        return np.sum(inputs)

class WeightedSum(Layer):

    def initial_parameters(self):
        a = Parameter('a', shape=(1,))
        b = Parameter('b', shape=(1,))
        return Phi(a, b)

    def evaluate(self, input1, input2):
        # Only two inputs are supported, and they are weighted differently.
        a, b = self.get_parameter_values('a', 'b')
        return a*input1 + b*input2

Now, let's create some instances of our new `Layers` and take a look at a summary
of their properties.

In [5]:
sum = Sum()
sum


Sum(shape=None)

In [9]:
weighted = WeightedSum()
weighted

WeightedSum(shape=None)
Parameter(name=a, shape=(1,))
----------------
Parameter(name=b, shape=(1,))

Notice that the two `Parameters` we created for the `WeightedSum` subclass
show up here. If we want to see more detail, like their numeric values,
we can `print` a full report for the `Layer` instead:

In [10]:
print(weighted)

WeightedSum(shape=None)
.parameters:

Parameter(name=a, shape=(1,))
----------------
.prior:     Normal(μ=array(shape=(1)), σ=array(shape=(1)))
.bounds:    (-inf, inf)
.is_frozen: False
.values:
[0.]
----------------
Index: 0
----------------

Parameter(name=b, shape=(1,))
----------------
.prior:     Normal(μ=array(shape=(1)), σ=array(shape=(1)))
.bounds:    (-inf, inf)
.is_frozen: False
.values:
[0.]
----------------
Index: 0
----------------



We can see that both parameters currently have a value of 0. This is because
the default behavior for parameter intialization is to set the values to the
mean of each `Parameter`'s prior `Distribution`. Since we didn't specify the
`Distribution` ourselves, it defaulted to a normal distribution with mean zero
and standard deviation 1. To choose a different prior, we could have specified
it in the `initial_parameters` method, or we can set it when creating a new
`WeightedSum` instance:

In [13]:
from nems.distributions import HalfNormal

weighted = WeightedSum(priors={'a': HalfNormal(sd=1)})
print(weighted)

WeightedSum(shape=None)
.parameters:

Parameter(name=a, shape=(1,))
----------------
.prior:     Normal(μ=array(shape=(1)), σ=array(shape=(1)))
.bounds:    (-inf, inf)
.is_frozen: False
.values:
[0.]
----------------
Index: 0
----------------

Parameter(name=b, shape=(1,))
----------------
.prior:     Normal(μ=array(shape=(1)), σ=array(shape=(1)))
.bounds:    (-inf, inf)
.is_frozen: False
.values:
[0.]
----------------
Index: 0
----------------

